In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing.data import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing


columns = ['dataset','building','has_act','start','end'] + range(122)
data = pd.read_csv('windat_dish_washer.csv', names=columns, index_col=None, 
                   header=None, parse_dates=['start','end'], date_parser=pd.to_datetime)
data.head()

,dataset,building,has_act,start,end,0,1,2,3,4,...,112,113,114,115,116,117,118,119,120,121
0,redd,1,1,2011-04-18 20:16:55,2011-04-18 22:16:55,458.230011,444.719971,411.690002,411.739990,405.639984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,redd,1,1,2011-04-18 20:17:52,2011-04-18 22:17:52,458.230011,444.719971,411.690002,411.739990,405.639984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,redd,1,1,2011-04-18 20:19:42,2011-04-18 22:19:42,444.719971,411.690002,411.739990,405.639984,409.779999,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,redd,1,1,2011-04-18 20:21:42,2011-04-18 22:21:42,411.690002,411.739990,405.639984,409.779999,241.640015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,redd,1,1,2011-04-18 20:23:06,2011-04-18 22:23:06,411.739990,405.639984,409.779999,241.640015,242.950012,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [26]:
# separar por dataset
data_redd = data[data['dataset'] == 'redd']
data_ukdale = data[data['dataset'] == 'ukdale']

# separar por si tienen o no activaciones
data_redd_act = data_redd[data_redd['has_act'] == 1]
data_redd_noact = data_redd[data_redd['has_act'] == 0]
data_ukdale_act = data_ukdale[data_ukdale['has_act'] == 1]
data_ukdale_noact = data_ukdale[data_ukdale['has_act'] == 0]

print "Casas de REDD con activaciones: " + str(data_redd_act['building'].unique())
print "Casas de REDD sin activaciones: " + str(data_redd_noact['building'].unique())
print "Casas de UK-DALE con activaciones: "+ str(data_ukdale_act['building'].unique())
print "Casas de UK-DALE sin activaciones: " + str(data_ukdale_noact['building'].unique())

Casas de REDD con activaciones: [1 2 3 4 5 6]
Casas de REDD sin activaciones: [2 3 4 5 6]
Casas de UK-DALE con activaciones: [1 2 5]
Casas de UK-DALE sin activaciones: []


Elijo la casas de REDD 5 y 6 para validar como casas no vistas.
No valido entrenando con un conjunto y validando con otro porque UK-DALE no tiene casas sin activaciones en este caso.

In [27]:
data_train = data_redd_act[data_redd['building'].isin([1,2,3,4])]
data_train = data_train.append(data_ukdale, ignore_index=True)
data_valid = data_redd_act[data_redd['building'].isin([5,6])]

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [28]:
data_train = data_train[range(122)]
data_train = data_train.fillna(0)
data_valid = data_valid[range(122)]
data_valid = data_valid.fillna(0)

d = np.shape(data_train)[1]/2
X_train = preprocessing.scale(np.array(data_train.iloc[:,:d]), with_mean=False, with_std=True)
y_train = preprocessing.scale(np.array(data_train.iloc[:,d:]), with_mean=False, with_std=True)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

X_valid = preprocessing.scale(np.array(data_valid.iloc[:,:d]), with_mean=False, with_std=True)
y_valid = preprocessing.scale(np.array(data_valid.iloc[:,d:]), with_mean=False, with_std=True)

In [29]:
regressor = MLPRegressor(solver='adam',
                        alpha=1e-5,
                        activation = 'tanh',
                        verbose = False,
                        hidden_layer_sizes=(500,500,500,),
                        tol=1e-8,
                        learning_rate='adaptive',
                        learning_rate_init=0.001,
                        random_state=1,
                        early_stopping=False,
                        validation_fraction=0.2)

regressor = regressor.fit(X_train, y_train)

In [34]:
print "Train data: score= {}".format(regressor.score(X_train, y_train))
print "Test data: score= {}".format(regressor.score(X_test, y_test))
print "Validate data: score= {}".format(regressor.score(X_valid, y_valid))

Train data: score= 0.99548791363
Test data: score= 0.414074805414
Validate data: score= -1.14594308552
